In [1]:
#!jupyter nbconvert --to=python GPPd_python3.ipynb

# After changes in the script using Jupyter notebook, save the notebook, run this line, and clear the output.

In [ ]:
class GPPd_global:
    def __init__(self,runname,resultpath,savepath,modulepath,meshpath,first_year,last_year):
        self.runname=runname
        self.resultpath=resultpath
        self.savepath=savepath
        self.modulepath=modulepath
        self.meshpath=meshpath
        self.first_year=first_year
        self.last_year=last_year
        
    def plot(self):
        import sys
        sys.path.append('../../../modules')
        
        import matplotlib.pyplot as plt
        from mpl_toolkits.basemap import Basemap
        from pylab import *
        from load_mesh_data_new import *
        import numpy as np
        from netCDF4 import Dataset
        from PiecewiseNorm import PiecewiseNorm
        import colormaps as cmaps
        from matplotlib.colors import ListedColormap

        viridis = ListedColormap(cmaps.viridis.colors)

        global_plot = True
        arctic_plot = False
        
        runid='fesom'

        years  = np.arange(self.first_year,self.last_year+1,1)
        months = np.linspace(0,11,12).astype(int)
        
        mesh = load_mesh(self.meshpath,abg=[0,0,0],get3d=True)
        
        var_id ='GPPd'

        GPP_Diatoms = np.zeros(len(mesh.x2))
        for ind in range(0,len(years)):
            print(years[ind])
            ncfile      = self.resultpath+var_id+'_'+runid+'_'+str(years[ind])+'0101'+'.nc'
            f           = Dataset(ncfile, 'r')    
            gppd        = f.variables[var_id][:].mean(axis=0)
            GPP_Diatoms = GPP_Diatoms + 12.01*gppd[0:mesh.n2d]
        dataGPPd = GPP_Diatoms/len(years)  
        
        dpicnt = 200
        if dpicnt == 600:
            xlablesize   = 12
            ylabelsize   = 12
            titlesize    = 14
            shrinkfactor = 0.9
        elif dpicnt == 200:
            xlablesize   = 8
            ylabelsize   = 8
            titlesize    = 10
            shrinkfactor = 0.9 
            
        contoursGPPd=np.array([0,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700])
        
        figGPPd = plt.figure(num=6, figsize=(8, 8), dpi=dpicnt, facecolor='w', edgecolor='k')
        elem2=mesh.elem[mesh.no_cyclic_elem,:]
        d=dataGPPd[elem2].mean(axis=1)
        k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
        elem2=elem2[k,:]
        print('ftriplot, number of dummy points:', len(d)-len(k))    
        map = Basemap(projection='robin',lon_0=0)
        x, y = map(mesh.x2, mesh.y2)
        map.drawmapboundary(fill_color='0.9')
        map.drawcoastlines()
        mlabels=[False,False,False,False]
        plabels=[True,True,True,True]		    
        map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
        map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]
        eps=(contoursGPPd.max()-contoursGPPd.min())/50.
        imGPPd=plt.tricontourf(x, y, elem2, dataGPPd, levels=contoursGPPd, cmap=viridis, norm=PiecewiseNorm(contoursGPPd), extend='max')
        labelGPPd = self.runname+': Diatom GPP ('+str(years[0])+' to '+str(years[len(years)-1])+')'
        plt.title(labelGPPd)
        cbarGPPd=map.colorbar(imGPPd,"bottom", size="5%", pad="2%")
        cbarGPPd.set_label(r'[mg C m$^{-2}$ day$^{-1}$]') 
        plt.savefig(self.savepath+self.runname+'_'+'GPPdspatial'+'_'+str(years[0])+'to'+str(years[len(years)-1])+'.png', dpi = dpicnt, bbox_inches='tight')
        plt.show(block=False)